<a href="https://colab.research.google.com/github/pbluc3m/collab-R/blob/main/procesar_tfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Parte 0: cojo los ficheros de Olivia y Teresa y los uno para lo que me interesa

import pandas as pd

input_teresa = 'JASPER_TFE_EPS_23-24.xlsx'
input_olivia = 'JASPER_TFF_COLME_23-24.xls'

df_teresa = pd.read_excel(input_teresa)
df_olivia = pd.read_excel(input_olivia)

columnas = ["NIA", "ALUMNO", "COD.CENTRO", "CENTRO", "COD.EST", "ESTUDIOS", "COD.PLAN", "PLAN", "COD.ASIG", "ASIGNATURA", "COD.OFERTA", "TITULO", "IDIOMA", "NOTA", "CALIF", "SUPERADA", "TUTOR", "DEPARTAMENTO_TUTOR", "TUTOR_EXTERNO"]
df_teresa_c = df_teresa.loc[:, df_teresa.columns.intersection(columnas)]
df_olivia_c = df_olivia.loc[:, df_olivia.columns.intersection(columnas)]

df = pd.concat([df_teresa_c, df_olivia_c], ignore_index=True)



In [4]:
# Vamos a arreglar dos casos identificados

# Asignar tutores a departamento
df.loc[df['TUTOR'] == "GONZALEZ CABAÑAS, JOSE", 'DEPARTAMENTO_TUTOR'] = "Departamento de Ingeniería Telemática"
df.loc[df['TUTOR'] == "GOMEZ MARTIN, ADRIAN", 'DEPARTAMENTO_TUTOR'] = "Departamento de Bioingeniería"

# Para doble grado Informatica ADE,
df['ASIGNATURA'] = df['ASIGNATURA'].replace(
    "Trabajo Fin de Grado (Informática)",
    "Trabajo Fin de Grado"
)
# La tilde
df['ASIGNATURA'] = df['ASIGNATURA'].replace(
    "Trabajo Fin de Master",
    "Trabajo Fin de Máster"
)
# La 'de'
df['ASIGNATURA'] = df['ASIGNATURA'].replace(
    "Trabajo de Fin de Máster",
    "Trabajo Fin de Máster"
)

In [5]:
# PARTE 1: BASIC STATS

total_rows = df.shape[0]
print(f"Total TFEs: {total_rows}")

# Vamos a quedarnos con los superados
df = df[df["SUPERADA"].str.strip().str.upper() == "S"]
filtered_rows = df.shape[0]
print(f"Total TFEs superados: {filtered_rows}")

# Vamos a quitar los de ADE
df = df[df["ASIGNATURA"] != "Trabajo Fin de Grado (ADE)"]

filtered_rows = df.shape[0]
print(f"Total TFEs superados sin ADE: {filtered_rows}")


Total TFEs: 1305
Total TFEs superados: 1305
Total TFEs superados sin ADE: 1305


In [6]:
# Por centro
centro = df.groupby("ASIGNATURA").size()
print(centro)

ASIGNATURA
Trabajo Fin de Grado     1031
Trabajo Fin de Máster     274
dtype: int64


In [8]:
# Por centro
centro = df.groupby("CENTRO").size()
print(centro)

CENTRO
Escuela Politécnica Superior (Colmenarejo)             79
Escuela Politécnica Superior (Leganés)                952
Escuela Politécnica Superior - Centro de Postgrado    274
dtype: int64


In [10]:
# Por estudios
estudio_counts = df.groupby("ESTUDIOS").size()
print(estudio_counts)

DEPARTAMENTO_TUTOR
Departamento de Bioingeniería                                               41
Departamento de Ciencia e Ingeniería de Materiales e Ingenieria Química     20
Departamento de Economía de la Empresa                                       1
Departamento de Estadística                                                 17
Departamento de Física                                                      10
Departamento de Informática                                                151
Departamento de Ingeniería Aeroespacial                                     66
Departamento de Ingeniería Eléctrica                                        59
Departamento de Ingeniería Mecánica                                         98
Departamento de Ingeniería Telemática                                       83
Departamento de Ingeniería Térmica y de Fluidos                             92
Departamento de Ingeniería de Sistemas y Automática                         57
Departamento de Matemáticas      

In [12]:
# Por departamento
departamento = df.groupby("DEPARTAMENTO_TUTOR").size()
print(departamento)

DEPARTAMENTO_TUTOR
Departamento de Bioingeniería                                               41
Departamento de Ciencia e Ingeniería de Materiales e Ingenieria Química     20
Departamento de Economía de la Empresa                                       1
Departamento de Estadística                                                 17
Departamento de Física                                                      10
Departamento de Informática                                                151
Departamento de Ingeniería Aeroespacial                                     66
Departamento de Ingeniería Eléctrica                                        59
Departamento de Ingeniería Mecánica                                         98
Departamento de Ingeniería Telemática                                       83
Departamento de Ingeniería Térmica y de Fluidos                             92
Departamento de Ingeniería de Sistemas y Automática                         57
Departamento de Matemáticas      

In [ ]:
# PARTE 2: SOLO "DEPARTAMENTO DE" Y GENERAMOS UN ARCHIVO POR DEPARTAMENTO
import unicodedata

def remove_accents(text):
    if isinstance(text, str):
        return "".join(c for c in unicodedata.normalize("NFD", text) if unicodedata.category(c) != "Mn")
    return text

input_file_clean = 'JASPER_TOTAL-23-24-clean.xlsx'
df = pd.read_excel(input_file_clean)

df["DEPARTAMENTO_TUTOR"] = df["DEPARTAMENTO_TUTOR"].apply(remove_accents)

# Estadisticas
total_rows = df.shape[0]
print(f"Total number of rows: {total_rows}")

filtered_df = df[df["DEPARTAMENTO_TUTOR"].str.startswith("Departamento de", na=False)]
total_rows = filtered_df.shape[0]
print(f"Total number of rows where Departamento de: {total_rows}")


output_folder = "TFE-Departamentos"
os.makedirs(output_folder, exist_ok=True)

for department, group_df in filtered_df.groupby("DEPARTAMENTO_TUTOR"):
    safe_filename = f"{department}.xlsx".replace("/", "_").replace(" ", "_")
    file_path = os.path.join(output_folder, safe_filename)
    group_df.to_excel(filename, index=False, engine="openpyxl")
    print(f"Saved {filename}")


Total number of rows: 1339
Total number of rows where Departamento de: 1332
Saved Departamento_de_Bioingenieria.xlsx
Saved Departamento_de_Ciencia_e_Ingenieria_de_Materiales_e_Ingenieria_Quimica.xlsx
Saved Departamento_de_Economia_de_la_Empresa.xlsx
Saved Departamento_de_Estadistica.xlsx
Saved Departamento_de_Fisica.xlsx
Saved Departamento_de_Informatica.xlsx
Saved Departamento_de_Ingenieria_Aeroespacial.xlsx
Saved Departamento_de_Ingenieria_Electrica.xlsx
Saved Departamento_de_Ingenieria_Mecanica.xlsx
Saved Departamento_de_Ingenieria_Telematica.xlsx
Saved Departamento_de_Ingenieria_Termica_y_de_Fluidos.xlsx
Saved Departamento_de_Ingenieria_de_Sistemas_y_Automatica.xlsx
Saved Departamento_de_Matematicas.xlsx
Saved Departamento_de_Mecanica_de_Medios_Continuos_y_Teoria_de_Estructuras.xlsx
Saved Departamento_de_Tecnologia_Electronica.xlsx
Saved Departamento_de_Teoria_de_la_Senal_y_Comunicaciones.xlsx
